<a href="https://colab.research.google.com/github/Kathy42xu/DL_TA/blob/main/TryMovies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##MovieLens32M
It contains 32000204 ratings and 2000072 tag applications across 87585 movies

Each user is represented by an id, and no other information is provided.



In [1]:
import os

os.makedirs('movielens', exist_ok=True)


!wget -O movielens/ml-32m.zip https://files.grouplens.org/datasets/movielens/ml-32m.zip

!unzip movielens/ml-32m.zip -d movielens/


--2024-12-27 23:23:51--  https://files.grouplens.org/datasets/movielens/ml-32m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 238950008 (228M) [application/zip]
Saving to: ‘movielens/ml-32m.zip’

movielens/ml-32m.zi 100%[===================>] 227.88M  33.6MB/s    in 7.2s    

2024-12-27 23:23:59 (31.7 MB/s) - ‘movielens/ml-32m.zip’ saved [238950008/238950008]

Archive:  movielens/ml-32m.zip
   creating: movielens/ml-32m/
  inflating: movielens/ml-32m/tags.csv  
  inflating: movielens/ml-32m/links.csv  
  inflating: movielens/ml-32m/README.txt  
  inflating: movielens/ml-32m/checksums.txt  
  inflating: movielens/ml-32m/ratings.csv  
  inflating: movielens/ml-32m/movies.csv  


#Inspect datafile


In [2]:
import pandas as pd

# Load the ratings data
ratings = pd.read_csv('movielens/ml-32m/ratings.csv')
print("Ratings Data:")
print(ratings.head())

# Load the movies data
movies = pd.read_csv('movielens/ml-32m/movies.csv')
print("\nMovies Data:")
print(movies.head())

# Load the tags data
tags = pd.read_csv('movielens/ml-32m/tags.csv')
print("\nTags Data:")
print(tags.head())

# Load the links data
links = pd.read_csv('movielens/ml-32m/links.csv')
print("\nLinks Data:")
print(links.head())



Ratings Data:
   userId  movieId  rating  timestamp
0       1       17     4.0  944249077
1       1       25     1.0  944250228
2       1       29     2.0  943230976
3       1       30     5.0  944249077
4       1       32     5.0  943228858

Movies Data:
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  

Tags Data:
   userId  movieId          tag   timestamp
0      22    26479  Kevin Kline  1583038886
1      22    79592     misogyny  1581476297
2    

In [3]:
# Get basic information about each dataset
print("\nRatings Summary:")
print(ratings.info())
print(ratings.describe())

print("\nMovies Summary:")
print(movies.info())

print("\nTags Summary:")
print(tags.info())

print("\nLinks Summary:")
print(links.info())



Ratings Summary:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32000204 entries, 0 to 32000203
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 976.6 MB
None
             userId       movieId        rating     timestamp
count  3.200020e+07  3.200020e+07  3.200020e+07  3.200020e+07
mean   1.002785e+05  2.931861e+04  3.540396e+00  1.275241e+09
std    5.794905e+04  5.095816e+04  1.058986e+00  2.561630e+08
min    1.000000e+00  1.000000e+00  5.000000e-01  7.896520e+08
25%    5.005300e+04  1.233000e+03  3.000000e+00  1.051012e+09
50%    1.002970e+05  3.452000e+03  3.500000e+00  1.272622e+09
75%    1.504510e+05  4.419900e+04  4.000000e+00  1.503158e+09
max    2.009480e+05  2.927570e+05  5.000000e+00  1.697164e+09

Movies Summary:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87585 entries, 0 to 87584
Data columns (to

Merge


In [4]:
merged_data = ratings.merge(movies, on='movieId')

# Display the first few rows of the merged dataset
print(merged_data.head())

   userId  movieId  rating  timestamp  \
0       1       17     4.0  944249077   
1       1       25     1.0  944250228   
2       1       29     2.0  943230976   
3       1       30     5.0  944249077   
4       1       32     5.0  943228858   

                                               title  \
0                       Sense and Sensibility (1995)   
1                           Leaving Las Vegas (1995)   
2  City of Lost Children, The (Cité des enfants p...   
3  Shanghai Triad (Yao a yao yao dao waipo qiao) ...   
4          Twelve Monkeys (a.k.a. 12 Monkeys) (1995)   

                                   genres  
0                           Drama|Romance  
1                           Drama|Romance  
2  Adventure|Drama|Fantasy|Mystery|Sci-Fi  
3                             Crime|Drama  
4                 Mystery|Sci-Fi|Thriller  


# time downsampling


In [22]:
import datetime


merged_data['year'] = pd.to_datetime(merged_data['timestamp'], unit='s').dt.year


all_ratings = merged_data[merged_data['year'] >= 2018]

print(f"amount of data after downsampling: {len(all_ratings)}")


amount of data after downsampling: 7411754


In [23]:
all_ratings['rating'].describe()

,rating
count,7.411754e+06
mean,3.571785e+00
std,1.057157e+00
min,5.000000e-01
25%,3.000000e+00
50%,3.500000e+00
75%,4.500000e+00
max,5.000000e+00


In [25]:
all_ratings['userId'].describe()
#total_user_id = len(all_ratings['userId'].unique())
#print(total_user_id)

,userId
count,7.411754e+06
mean,1.003554e+05
std,5.797824e+04
min,8.000000e+00
25%,5.028000e+04
50%,1.001320e+05
75%,1.505690e+05
max,2.009450e+05


# following code from solution


In [26]:
all_ratings.head()

,userId,movieId,rating,timestamp,title,genres,year
470,8,32,4.0,1553765549,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller,2019
471,8,47,5.0,1553765552,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,2019
472,8,260,4.0,1553765477,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,2019
473,8,296,4.5,1553765465,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,2019
474,8,527,4.0,1553765389,Schindler's List (1993),Drama|War,2019


In [27]:
total_item_id = len(all_ratings['movieId'].unique())
print(total_item_id)

79293


In [28]:
total_item_id = len(all_ratings['userId'].unique())
print(total_item_id)

43848
